In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# Chat model instance

In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

/Users/albertochiavetta/PycharmProjects/GenAI_lesson/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Esempio di allucinazione
llm.invoke("What is it today?")

AIMessage(content="Today's date is October 3, 2023.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 12, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CHV56XwjAYMRxJMhpJKpu7E4Ik6y3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d7572516-cd71-4aeb-af2c-e6015ee923a9-0', usage_metadata={'input_tokens': 12, 'output_tokens': 11, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [26]:
question = "Che score ha avuto gpt-5 nei benchmark?"

In [5]:
# Esempio di ammissione di ignoranza
llm.invoke(question)

AIMessage(content='Non ho informazioni specifiche sui punteggi di GPT-5 nei benchmark poiché il mio addestramento include dati fino a ottobre 2021 e non sono aggiornato sugli sviluppi successivi o sui modelli rilasciati dopo quella data. Se desideri dettagli sui benchmark di nuovi modelli, ti consiglio di controllare le fonti ufficiali di OpenAI o altre pubblicazioni recenti che potrebbero avere le informazioni più aggiornate.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 18, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CHV5WfKIibsaf9nRVQW4bpde7m02D', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--5566914e-5390-4

# RAG PIPELINE

## 1. Indexing (ingestion) -> building the knowledge base

### Loading documents

#### From web url

In [3]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
web_paths = ("https://openai.com/it-IT/index/introducing-gpt-5/",)

#bs4_strainer = bs4.SoupStrainer(name="article")
loader = WebBaseLoader(
    web_paths=web_paths,
    #bs_kwargs={"parse_only": bs4_strainer},
    #bs_get_text_kwargs={"separator": " | ", "strip":True},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

Total characters: 35549


In [5]:
docs[0].metadata

{'source': 'https://openai.com/it-IT/index/introducing-gpt-5/',
 'title': 'Ti presentiamo GPT-5 | OpenAI',
 'description': "È in assoluto il nostro modello più intelligente, veloce ed efficace, con capacità di ragionamento integrata che mette l'intelligenza di livello esperto a disposizione di tutti.",
 'language': 'it-IT'}

In [6]:
docs[0].page_content

'Ti presentiamo GPT-5 | OpenAIVai al contenuto principaleAccediPassa aChatGPT(si apre in una nuova finestra)Sora(si apre in una nuova finestra)API Platform(si apre in una nuova finestra)RicercaSicurezzaPer le aziendePiattaforma APIChatGPTSoraStorieAziendaNotizieRicercaTorna al menu principaleIndice delle ricerchePanoramica delle ricercheResidenza di ricercaAggiornamenti più recentiGPT-5OpenAI o3 e o4-miniGPT-4.5OpenAI o1GPT-4oSoraSicurezzaTorna al menu principaleApproccio alla sicurezzaSicurezza e privacyPer le aziendeTorna al menu principalePanoramica aziendeStartupSoluzioniTariffe di ChatGPTTariffe APIContatta l’Ufficio venditePiattaforma APITorna al menu principalePanoramica della piattaformaTariffeCodexOpen ModelsForum degli sviluppatori(si apre in una nuova finestra)ChatGPTTorna al menu principaleEsplora ChatGPTBusinessEnterpriseIstruzioneTariffeScaricaSoraTorna al menu principalePanoramica di SoraFunzioniTariffeCentro assistenza(si apre in una nuova finestra)Accesso a Sora(si apr

### Splitting documents into chunks

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
docs_splits = text_splitter.split_documents(docs)

print(f"Split web page into {len(docs_splits)} sub-documents.")

Split web page into 45 sub-documents.


In [8]:
docs_splits[0]

Document(metadata={'source': 'https://openai.com/it-IT/index/introducing-gpt-5/', 'title': 'Ti presentiamo GPT-5 | OpenAI', 'description': "È in assoluto il nostro modello più intelligente, veloce ed efficace, con capacità di ragionamento integrata che mette l'intelligenza di livello esperto a disposizione di tutti.", 'language': 'it-IT', 'start_index': 0}, page_content='Ti presentiamo GPT-5 | OpenAIVai al contenuto principaleAccediPassa aChatGPT(si apre in una nuova finestra)Sora(si apre in una nuova finestra)API Platform(si apre in una nuova finestra)RicercaSicurezzaPer le aziendePiattaforma APIChatGPTSoraStorieAziendaNotizieRicercaTorna al menu principaleIndice delle ricerchePanoramica delle ricercheResidenza di ricercaAggiornamenti più recentiGPT-5OpenAI o3 e o4-miniGPT-4.5OpenAI o1GPT-4oSoraSicurezzaTorna al menu principaleApproccio alla sicurezzaSicurezza e privacyPer le aziendeTorna al menu principalePanoramica aziendeStartupSoluzioniTariffe di ChatGPTTariffe APIContatta l’Uffic

In [9]:
html_docs = loader.scrape()
html_docs

<!DOCTYPE html>
<html dir="ltr" lang="it-IT" style="--document-width:100dvw;--nav-width:200px;--content-width:calc(var(--document-width) - var(--nav-width));--gutter-size:max(20px, calc((var(--document-width) - 68rem) / 2));--media-gutter-size:max(20px, calc((var(--document-width) - 1728px) / 2))"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><link data-precedence="next" href="/_next/static/css/4105e979b7485bff.css" rel="stylesheet"/><link data-precedence="next" href="/_next/static/css/ba2e6b7c939be632.css" rel="stylesheet"/><link data-precedence="next" href="/_next/static/css/1be87e885847316b.css" rel="stylesheet"/><link data-precedence="next" href="/_next/static/css/06f64a967f7d7746.css" rel="stylesheet"/><link data-precedence="next" href="/_next/static/css/795fe67cff874c18.css" rel="stylesheet"/><link data-precedence="next" href="/_next/static/css/e44a6bc6479102dd.css" rel="stylesheet"/><link data-precedence="next" href="/_next/sta

In [9]:
from langchain_text_splitters import HTMLHeaderTextSplitter

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on)

html_splits = html_splitter.split_text(str(html_docs))

In [10]:
html_splits

[Document(metadata={}, page_content="$! /$ $! /$  \nVai al contenuto principale  \nAccedi  \nPassa a  \nChatGPT  \n(si apre in una nuova finestra)  \nSora  \n(si apre in una nuova finestra)  \nAPI Platform  \n(si apre in una nuova finestra)  \nRicerca  \nSicurezza  \nPer le aziende  \nPiattaforma API  \nChatGPT  \nSora  \nStorie  \nAzienda  \nNotizie  \n$ /$  \nRicerca  \nTorna al menu principale  \nIndice delle ricerche  \nPanoramica delle ricerche  \nResidenza di ricerca  \nAggiornamenti più recenti  \nGPT-5  \nOpenAI o3 e o4-mini  \nGPT-4.5  \nOpenAI o1  \nGPT-4o  \nSora  \nSicurezza  \nTorna al menu principale  \nApproccio alla sicurezza  \nSicurezza e privacy  \nPer le aziende  \nTorna al menu principale  \nPanoramica aziende  \nStartup  \nSoluzioni  \nTariffe di ChatGPT  \nTariffe API  \nContatta l’Ufficio vendite  \nPiattaforma API  \nTorna al menu principale  \nPanoramica della piattaforma  \nTariffe  \nCodex  \nOpen Models  \nForum degli sviluppatori  \n(si apre in una nuova f

### Store documents

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="paraphrase-multilingual-mpnet-base-v2")

In [11]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embedding=embedding_model)

In [12]:
chunks_ids = vector_store.add_documents(documents=docs_splits)

In [13]:
vector_store.get_by_ids([chunks_ids[0]])

[Document(id='42db5d9a-0776-4d7b-a084-5dd533490660', metadata={'source': 'https://openai.com/it-IT/index/introducing-gpt-5/', 'title': 'Ti presentiamo GPT-5 | OpenAI', 'description': "È in assoluto il nostro modello più intelligente, veloce ed efficace, con capacità di ragionamento integrata che mette l'intelligenza di livello esperto a disposizione di tutti.", 'language': 'it-IT', 'start_index': 0}, page_content='Ti presentiamo GPT-5 | OpenAIVai al contenuto principaleAccediPassa aChatGPT(si apre in una nuova finestra)Sora(si apre in una nuova finestra)API Platform(si apre in una nuova finestra)RicercaSicurezzaPer le aziendePiattaforma APIChatGPTSoraStorieAziendaNotizieRicercaTorna al menu principaleIndice delle ricerchePanoramica delle ricercheResidenza di ricercaAggiornamenti più recentiGPT-5OpenAI o3 e o4-miniGPT-4.5OpenAI o1GPT-4oSoraSicurezzaTorna al menu principaleApproccio alla sicurezzaSicurezza e privacyPer le aziendeTorna al menu principalePanoramica aziendeStartupSoluzioniT

In [16]:
len(vector_store.store.get(chunks_ids[0])['vector'])

768

## 2. Retrieval and Generation

### Semantic Search Example

In [17]:
res = vector_store.similarity_search_with_score("Che score ha avuto gpt-5 nei benchmark?")

In [19]:
for chunk in res:
    print("score: ", chunk[1])
    print("chunk: ", chunk[0].page_content)
    print("\n")

score:  0.5814943028130202
chunk:  sul nostro approccio rigoroso alla sicurezza per GPT‑5, consulta la scheda di sistema.GPT‑5 ProPer le attività più impegnative e complesse, stiamo anche rilasciando GPT‑5 pro, che sostituisce OpenAI o3‑pro. Si tratta di una variante di GPT‑5 che pensa ancora più a lungo, utilizzando un calcolo parallelo scalabile ma efficiente in fase di test, per garantire risposte della massima qualità e completezza. GPT‑5 pro raggiunge le prestazioni più elevate nella famiglia GPT‑5 in diversi benchmark di intelligenza complessi, tra cui gli ottimi risultati ottenuti su GPQA, che prevedono domande di scienze estremamente difficili.Nelle valutazioni su oltre 1000 prompt di ragionamento reali e di valore economico, gli esperti esterni hanno preferito GPT‑5 pro rispetto alla “modalità di pensiero GPT‑5” nel 67,8% dei casi. GPT‑5 pro ha registrato il 22% in meno di errori gravi e ha ottenuto risultati eccellenti nei settori della salute, della scienza, della matematica

### Retrieve

In [24]:
retriever = vector_store.as_retriever(
    search_type='similarity',
    search_kwargs={
        "k": 5,
    }
)

#retriever = vector_store.as_retriever(
#    search_type='mmr',
#    search_kwargs={
#        "k": 5,
#        "fetch_k":20,
#        "lambda_mult":0.2
#    }
#)

In [27]:
retriever.invoke(question)

[Document(id='26362760-8226-420b-bb69-e76e8f115df8', metadata={'source': 'https://openai.com/it-IT/index/introducing-gpt-5/', 'title': 'Ti presentiamo GPT-5 | OpenAI', 'description': "È in assoluto il nostro modello più intelligente, veloce ed efficace, con capacità di ragionamento integrata che mette l'intelligenza di livello esperto a disposizione di tutti.", 'language': 'it-IT', 'start_index': 24020}, page_content='sul nostro approccio rigoroso alla sicurezza per GPT‑5, consulta la scheda di sistema.GPT‑5 ProPer le attività più impegnative e complesse, stiamo anche rilasciando GPT‑5 pro, che sostituisce OpenAI o3‑pro. Si tratta di una variante di GPT‑5 che pensa ancora più a lungo, utilizzando un calcolo parallelo scalabile ma efficiente in fase di test, per garantire risposte della massima qualità e completezza. GPT‑5 pro raggiunge le prestazioni più elevate nella famiglia GPT‑5 in diversi benchmark di intelligenza complessi, tra cui gli ottimi risultati ottenuti su GPQA, che preve

### Augmentation and Generation

#### Simple Generation

In [28]:
llm.invoke(question)

AIMessage(content='Fino a ottobre 2023, OpenAI non ha rilasciato informazioni specifiche sui punteggi o le prestazioni di GPT-5 in benchmark ufficiali. Pertanto, non posso fornire dettagli sui punteggi che GPT-5 ha ottenuto nei vari test di valutazione. Se desideri informazioni su GPT-3.5 o altre versioni precedenti, sarò felice di assisterti!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 18, 'total_tokens': 105, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CHVPlYVRp5mWdlBrOmMRWD8YbD1X2', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--bd4b63dd-786d-4299-a325-4a1121fdec02-0', usage_metadata={'input_tokens': 18, 'output_tokens': 87, 't

#### Retrieve augmented generation

In [29]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant that answers questions using the context provided.
Answer as precisely as possible using only the context.
If you truly cannot find the answer, just say that you don't know.


Context:
{context}

Question: {question}

Answer:"""

prompt_template = ChatPromptTemplate.from_template(template)

In [30]:
context = retriever.invoke(question)
context = "\n\n".join(d.page_content for d in context)

In [31]:
prompt = prompt_template.invoke({"question": question, "context": context})

In [32]:
prompt

ChatPromptValue(messages=[HumanMessage(content="You are an assistant that answers questions using the context provided.\nAnswer as precisely as possible using only the context.\nIf you truly cannot find the answer, just say that you don't know.\n\n\nContext:\nsul nostro approccio rigoroso alla sicurezza per GPT‑5, consulta la scheda di sistema.GPT‑5 ProPer le attività più impegnative e complesse, stiamo anche rilasciando GPT‑5 pro, che sostituisce OpenAI o3‑pro. Si tratta di una variante di GPT‑5 che pensa ancora più a lungo, utilizzando un calcolo parallelo scalabile ma efficiente in fase di test, per garantire risposte della massima qualità e completezza. GPT‑5 pro raggiunge le prestazioni più elevate nella famiglia GPT‑5 in diversi benchmark di intelligenza complessi, tra cui gli ottimi risultati ottenuti su GPQA, che prevedono domande di scienze estremamente difficili.Nelle valutazioni su oltre 1000 prompt di ragionamento reali e di valore economico, gli esperti esterni hanno prefe

In [33]:
llm.invoke(prompt)

AIMessage(content='GPT-5 ha ottenuto i seguenti punteggi nei benchmark:\n\n- 94,6% su AIME 2025 senza strumenti in matematica.\n- 74,9% su SWE-bench Verified e 88% su Aider Polyglot in programmazione nel mondo reale.\n- 84,2% su MMMU in comprensione multimodale.\n- 46,2% su HealthBench Hard in salute.\n- 88,4% su GPQA senza strumenti.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 1281, 'total_tokens': 1383, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CHVT1ZaN7WJo36p8Ko0DHE1Q95HKP', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--31bea5df-b168-4ca9-a0c5-e62605226bdd-0', usage_metadata={'input_tokens': 1281, 'output_tokens': 102, 'tot

### Rag chain

In [34]:
from langchain_core.runnables import RunnablePassthrough

In [36]:
context = retriever.invoke(question)
prompt = prompt_template.invoke({"question": question, "context": context})
llm.invoke(prompt)

In [35]:
rag_chain = (
    {"question": RunnablePassthrough(), "context": retriever}
    | prompt_template
    | llm
)
rag_chain.invoke(question)

AIMessage(content="GPT-5 ha ottenuto un punteggio del 94,6% su AIME 2025 senza strumenti, del 74,9% su SWE-bench Verified, dell'88% su Aider Polyglot, dell'84,2% su MMMU e del 46,2% su HealthBench Hard. Inoltre, ha registrato un punteggio dell'88,4% su GPQA senza strumenti.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 1912, 'total_tokens': 2002, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CHVWPBnaFehymjxykxClrRyZNNM0y', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7ef73293-bdc2-4e82-b8a0-da3862200f67-0', usage_metadata={'input_tokens': 1912, 'output_tokens': 90, 'total_tokens': 2002, 'input_token_details': {'audio': 0, 'cache_rea

In [37]:
from pydantic import BaseModel, Field

class BenchmarkResult(BaseModel):
    benchmark_name: str = Field(description="Name of the benchmark")
    score: float = Field(description="Score of the benchmark")

class ModelScore(BaseModel):
    model_name: str = Field(description="Name of the model")
    scores: list[BenchmarkResult] = Field(description="List of benchmark results")

In [38]:
llm.with_structured_output(ModelScore).invoke(prompt)

ModelScore(model_name='GPT-5', scores=[BenchmarkResult(benchmark_name='AIME 2025', score=94.6), BenchmarkResult(benchmark_name='SWE-bench Verified', score=74.9), BenchmarkResult(benchmark_name='Aider Polyglot', score=88.0), BenchmarkResult(benchmark_name='MMMU', score=84.2), BenchmarkResult(benchmark_name='HealthBench Hard', score=46.2), BenchmarkResult(benchmark_name='GPQA', score=88.4)])